In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#identify page to scrape
url = 'https://www.basketball-reference.com/leagues/NBA_2019_numbers.html'

In [3]:
#make request for the page
datapull = requests.get(url)

In [4]:
#parse all the HTML from the request
soup = BeautifulSoup(datapull.text, 'html.parser')

In [5]:
#subset the numbers & names data from code inspection
pull_uniforms = soup.find_all('table', class_='no_columns')

In [6]:
#set series list containers to merge
all_names = []
all_numbers = []
all_teams = []

In [7]:
#set iterator starters
u_starter=0
n_starter=0


#loop through uniform number & player name set
for i in pull_uniforms:
    
    #pull uniform number value
    number_pull = pull_uniforms[u_starter].find('caption', class_='poptip')
    number_value = number_pull.contents[0]
    
    #pull name and team anchor text
    names = pull_uniforms[u_starter].find_all('a')

    
    #loop separate name from team anchors
    for j in names:
        names_value = names[n_starter].contents[0]
        
        #if the text value is a name...
        if len(names[n_starter].contents[0]) > 5 :
            #add value to names list.
            all_names.append(names[n_starter].contents[0])
        #if the text value is a three-letter team...    
        elif len(names[n_starter].contents[0]) == 3:
            #add the team name and number to series, assuming the previous anchor was not a team name
            if len(names[n_starter].contents[0]) != len(names[n_starter - 1].contents[0]):
                all_teams.append(names[n_starter].contents[0])
                all_numbers.append(number_value)
            #if the previous anchor was a team name, copy the last recorded player name and add team and number
            else:
                all_names.append(all_names[-1])
                all_teams.append(names[n_starter].contents[0])
                all_numbers.append(number_value)
                                                                               

        #iterate to next item
        n_starter +=1

    #reset name interator
    n_starter = 0
    
    #next uniform set
    u_starter +=1

In [8]:
#assemble dictionary from both name and number series
name_number_dict = {'NAME': all_names, 'NUMBER': all_numbers, 'TEAM': all_teams}

In [9]:
#convert dictionary to pandas dataframe
df = pd.DataFrame.from_dict(name_number_dict)

In [10]:
#sort by name and reset index
df = df.sort_values(by=["NAME", "TEAM"])
df = df.reset_index(drop=True)

In [11]:
#inspection -- note: special charactered players at bottom values
df

,NAME,NUMBER,TEAM
0,Aaron Gordon,00,ORL
1,Aaron Holiday,3,IND
2,Abdel Nader,11,OKC
3,Al Horford,42,BOS
4,Al-Farouq Aminu,8,POR
...,...,...,...
617,Zaza Pachulia,27,DET
618,Zhaire Smith,8,PHI
619,Zhou Qi,9,HOU
620,Álex Abrines,8,OKC


In [12]:
#rename special charactered players
df.at[620,"NAME"] = "Alex Abrines"
df.at[621,"NAME"] = "Angel Delgado"

#sort & reset
df = df.sort_values(by=["NAME", "TEAM"])
df = df.reset_index(drop=True)

In [13]:
#export pandas dataframe to csv
df.to_csv('playernumbers.csv')